In [4]:
import cv2
import numpy as np

In [27]:
class Memo:
    def __init__(self, color, content, size=100):
        self.color = np.array(color, dtype=np.uint8)
        self.content = content
        self.size = int(size)
        
        self.pic = np.zeros((self.size, self.size, 3), dtype=np.uint8)
        self.font = cv2.FONT_HERSHEY_SIMPLEX
        self.font_scale = 1.5
        self.font_color = (255, 255, 255)  # 白色
        self.font_thickness = 2

        self.update_pic()

    def update_pic(self):
        picture = np.broadcast_to(self.color, (self.size, self.size, 3))
        picture = cv2.cvtColor(picture, cv2.COLOR_BGR2RGB)
        
        # 获取文本的大小
        text_size = cv2.getTextSize(self.content, self.font, self.font_scale, self.font_thickness)

        # 计算文本的位置
        text_x = int((self.size - text_size[0][0]) / 2)
        text_y = int((self.size + text_size[0][1]) / 2)
        
        # 在图像上绘制文本
        cv2.putText(picture, self.content, (text_x, text_y), self.font, self.font_scale, self.font_color, self.font_thickness)

        # update
        self.pic = picture.astype(np.uint8)

    def get_pic(self):
        return self.pic

    def merge(self, memo):
        self.content = self.content + memo.content
        self.color = (self.color + memo.color) // 2
        self.update_pic()

        

In [31]:
memo1 = Memo([200, 30, 30], "A")
memo2 = Memo([30, 30, 200], "B")

memo2.merge(memo1)

# 显示图像
cv2.imshow("memo1", memo1.get_pic())
cv2.imshow("memo2", memo2.get_pic())
cv2.waitKey(0)
cv2.destroyAllWindows()

In [1]:
import cv2

# 调用笔记本摄像头
cap = cv2.VideoCapture(0)

while True:
    # 读取摄像头捕获的帧
    ret, frame = cap.read()
    
    if ret:
        # 显示帧
        cv2.imshow("camera", frame)
    
    # 按 'q' 键退出循环
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 释放摄像头并关闭窗口
cap.release()
cv2.destroyAllWindows()
